In [ ]:
from langchain.schema import BaseOutputParser
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
import json


# --- Custom Output Parser for JSON ---


class GuardedJSONParser(BaseOutputParser):
    """Parses LLM output by looking for a guard token and parsing JSON safely."""

    def parse(self, text: str) -> dict:
        if "JSON_OUTPUT_START" in text:
            text = text.split("JSON_OUTPUT_START")[-1].strip()

        if not text.endswith("}"):
            text += "}"

        try:
            return json.loads(text)
        except json.JSONDecodeError as e:
            raise ValueError(f"Failed to parse JSON: {e}\nRaw output: {text}")


# --- Build the prompt ---

prompt_template = ChatPromptTemplate.from_template(
    """
You are an expert journalist and fact-checker.

Compare the extracted claim with the original post and answer five fact-checking questions.

For each question:
- Answer `true` or `false`.
- If `false`, list specific errors.
- If `true`, leave the errors list empty.

Respond ONLY in valid JSON format. End after the JSON. Do not explain anything.

Here is the required output format:

{{
  "questions": [
    {{
      "id": "meaning_preserved",
      "answer": true,
      "errors": []
    }},
    {{
      "id": "correct_named_entities",
      "answer": true,
      "errors": []
    }},
    {{
      "id": "correct_numbers",
      "answer": true,
      "errors": []
    }},
    {{
      "id": "no_added_data",
      "answer": true,
      "errors": []
    }},
    {{
      "id": "no_missing_data",
      "answer": true,
      "errors": []
    }}
  ]
}}

---

CLAIM:
\"\"\"{claim}\"\"\"

POST:
\"\"\"{post}\"\"\"

JSON_OUTPUT_START
"""
)

# --- Create the LLM ---

llm = ChatOpenAI(
    model="your-local-model-name",  # replace with your model name
    openai_api_base="http://localhost:1234/v1",
    openai_api_key="lm-studio",  # dummy key
    temperature=0.0,
    stop=["}"],  # stop at end of JSON
)

# --- Build the Chain ---

factcheck_chain = LLMChain(
    llm=llm, prompt=prompt_template, output_parser=GuardedJSONParser()
)

# --- Function to call ---


def factcheck_claim_against_post(claim: str, post: str) -> dict:
    result = factcheck_chain.run(claim=claim, post=post)
    return result


# --- Example usage ---

if __name__ == "__main__":
    claim = "The president of Denmark pledged 3 billion euros to Norway in 2025."
    post = "In 2025, the prime minister of Sweden pledged 1.5 billion euros to Finland."

    output = factcheck_claim_against_post(claim, post)
    print(json.dumps(output, indent=2))